In [0]:
import sys
import os

# Get the root directory dynamically
project_root = os.getcwd()
src_path = os.path.join(project_root, "src")

# Add "src" to sys.path if it's not already there
if src_path not in sys.path:
    sys.path.insert(0, src_path)

# Import module
from src.wt_ingestion import RawDataIngestor

# Get Databricks username dynamically
current_user = dbutils.notebook.entry_point.getDbutils().notebook().getContext().tags().get("user").get()

# Define DBFS path for data storage
user_dbfs_path = f"dbfs:/FileStore/{current_user}/ovidiu_toma_wind_turbine_project/data/"

# Define data directory path dynamically
if "DATABRICKS_RUNTIME_VERSION" in os.environ:
    raw_directory_path = user_dbfs_path  # Databricks path (DBFS)
else:
    raw_directory_path = os.path.abspath("data/")  # Local path for interviewers

print(f"Using data directory: {raw_directory_path}")

# Instantiate the ingestor
ingestor = RawDataIngestor(spark)

# Load and write data
df_original_turbine_data = ingestor.load_original_turbine_data(raw_directory_path)
ingestor.write_bronze(df_original_turbine_data, "original_turbine_bronze")

df_new_turbine_data = ingestor.load_new_turbine_data(raw_directory_path)
ingestor.write_bronze(df_new_turbine_data, "new_turbine_bronze")
